# Income Prediction

In [1]:
import os
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor

from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import tensorflow as tf
from tensorflow import keras

TARGET_VARIABLE = 'Total Yearly Income [EUR]'

### Data preprocessing & Feature extraction

Load training data

In [2]:
DATA_DIR = os.path.abspath('data')
training_data_filepath = os.path.join(DATA_DIR, 'tcd-ml-1920-group-income-train.csv')
df = pd.read_csv(training_data_filepath)

c:\users\fitze\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fix and Format data

In [3]:
def format_data(df):
    # Years are in order so can replace missing years with the last valid year before
    df['Year of Record'] = df['Year of Record'].fillna(method='ffill')

    # Replace invalid Housing situation with other
    df['Housing Situation'] = df['Housing Situation'].replace([0, '0', 'nA',np.nan], 'Unknown')

    # Convert years Work Experience to float and replace '#NUM!' entries with mean
    df['Work Experience in Current Job [years]'] = pd.to_numeric(df['Work Experience in Current Job [years]'], errors='coerce')
    df['Work Experience in Current Job [years]'].fillna(df['Work Experience in Current Job [years]'].mean(), inplace=True)

    # Replace Nan Satisfation with employer with unknown
    df['Satisfation with employer'].fillna('Unknown', inplace=True)

    # Remove ' EUR' from addition to salary and convert to float
    df['Yearly Income in addition to Salary (e.g. Rental Income)'].replace(' EUR$', '', regex=True, inplace=True)
    df['Yearly Income in addition to Salary (e.g. Rental Income)'] = pd.to_numeric(
        df['Yearly Income in addition to Salary (e.g. Rental Income)'], errors='raise')

    # Fix age values
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['Age'] = df['Age'].astype(int)

    # Fix gender values
    df['Gender'] = df['Gender'].replace(['0', 'unknown', np.NaN], 'other')
    df['Gender'] = df['Gender'].replace('f', 'female')

    # Fix university values
    df['University Degree'] = df['University Degree'].replace(['0', np.NaN], 'No')

    # Fix hair color values
    df['Hair Color'] = df['Hair Color'].replace(['0', np.NaN], 'Unknown')
    
    # Remove unwanted features
    del df['Instance']
    
    return df

df = format_data(df)

Encode Data

In [4]:
'''
def one_hot_encode_data(df, one_hot_encode):
    for feature in one_hot_encode:
        df = pd.concat([df,pd.get_dummies(df[feature], prefix=feature, drop_first=True, sparse=True)],axis=1)
        df.drop([feature],axis=1, inplace=True)
        
    return df

one_hot_encode = ['Housing Situation', 'Satisfation with employer', 
                  'Gender', 'University Degree', 'Hair Color']
df = one_hot_encode_data(df, one_hot_encode)
'''

def frequency_encode_data(df, target_encode, use_mappings={}):
    calculated_mappings = {}
    
    for feature in target_encode:
        feature_mappings = None  
        
        if use_mappings != {}:
            # Use previous mappings
            if feature not in use_mappings:
                ValueError('{} mappings is not in provided use_mappings'.format(feature))
            else:
                feature_mappings = use_mappings[feature]
        else:
            # Calculate new feature mappings
            feature_mappings = df.groupby(feature).size()
            
        # Store calculated mappings
        calculated_mappings[feature] = feature_mappings
                    
        df[f'{feature}_freq'] = df[feature].map(calculated_mappings[feature])
    return df, calculated_mappings


frequency_encode = ['Gender', 'University Degree', 'Country', 'Profession', 
                       'Housing Situation', 'Satisfation with employer']
df, frequency_encode_mappings = frequency_encode_data(df, frequency_encode)


def target_encode_data(df, target_encode, use_mappings={}):
    mean_target = df[TARGET_VARIABLE].mean()
    calculated_mappings = {}
    
    for feature in target_encode:
        feature_mappings = None
        
        if use_mappings != {}:
            # Use previous mappings
            if feature not in use_mappings:
                ValueError('{} mappings is not in provided use_mappings'.format(feature))
            else:
                feature_mappings = use_mappings[feature]
        else:
            # Calculate new feature mappings
            feature_mappings = df.groupby(feature)[TARGET_VARIABLE].mean()
            
        # Store calculated mappings
        calculated_mappings[feature] = feature_mappings
                    
        df[feature] = df[feature].map(calculated_mappings[feature])
        df[feature].fillna(mean_target, inplace=True)
        
    return df, calculated_mappings

target_encode = ['Country', 'Profession', 'Housing Situation', 'Satisfation with employer', 
                  'Gender', 'University Degree', 'Hair Color']
df, target_encode_mappings = target_encode_data(df, target_encode)

In [5]:
'''
# Log age and exp
import matplotlib.pyplot as plt
import seaborn as sns

df['Age_sq'] = np.square(df['Age'])
df['Age_log'] = np.log(df['Age'])

df['exp_sq'] = np.square(df['Work Experience in Current Job [years]'])
df['exp_log'] = np.log(df['Work Experience in Current Job [years]'].clip(lower=1))
'''


"\n# Log age and exp\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\ndf['Age_sq'] = np.square(df['Age'])\ndf['Age_log'] = np.log(df['Age'])\n\ndf['exp_sq'] = np.square(df['Work Experience in Current Job [years]'])\ndf['exp_log'] = np.log(df['Work Experience in Current Job [years]'].clip(lower=1))\n"

### Training

Standardize and Split data

In [6]:
from sklearn.preprocessing import StandardScaler

# Standardize data
scaler = StandardScaler()
scale_columns = df.columns.to_list()
scale_columns.remove(TARGET_VARIABLE)
df[scale_columns] = scaler.fit_transform(df[scale_columns])

# Split data
X = df.drop(columns=TARGET_VARIABLE).values
y = df[TARGET_VARIABLE].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Train Model

In [7]:
start_time = time.time()

model = CatBoostRegressor(iterations=6000)
model.fit(X_train, y_train)

training_time = time.time() - start_time
print('--- Training took {} sec ---'.format(training_time))

0:	learn: 109255.6662975	total: 118ms	remaining: 11m 49s
1:	learn: 106800.2944561	total: 184ms	remaining: 9m 11s
2:	learn: 104477.9666352	total: 242ms	remaining: 8m 4s
3:	learn: 102193.5088300	total: 299ms	remaining: 7m 28s
4:	learn: 100040.7876424	total: 369ms	remaining: 7m 22s
5:	learn: 97963.2283104	total: 431ms	remaining: 7m 10s
6:	learn: 95916.5648697	total: 490ms	remaining: 6m 59s
7:	learn: 93936.0127271	total: 554ms	remaining: 6m 54s
8:	learn: 92034.6926030	total: 620ms	remaining: 6m 52s
9:	learn: 90222.4137060	total: 678ms	remaining: 6m 45s
10:	learn: 88484.1676547	total: 744ms	remaining: 6m 45s
11:	learn: 86778.5491466	total: 813ms	remaining: 6m 45s
12:	learn: 85126.9452242	total: 870ms	remaining: 6m 40s
13:	learn: 83531.3586242	total: 928ms	remaining: 6m 36s
14:	learn: 81970.7489703	total: 998ms	remaining: 6m 38s
15:	learn: 80486.7223069	total: 1.06s	remaining: 6m 38s
16:	learn: 79043.5229470	total: 1.13s	remaining: 6m 36s
17:	learn: 77664.6378255	total: 1.19s	remaining: 6m 3

146:	learn: 33219.4789635	total: 9.85s	remaining: 6m 32s
147:	learn: 33120.3460330	total: 9.93s	remaining: 6m 32s
148:	learn: 33043.5410718	total: 9.99s	remaining: 6m 32s
149:	learn: 32977.3331882	total: 10.1s	remaining: 6m 32s
150:	learn: 32908.8775493	total: 10.1s	remaining: 6m 32s
151:	learn: 32831.2966336	total: 10.2s	remaining: 6m 32s
152:	learn: 32768.5625062	total: 10.2s	remaining: 6m 31s
153:	learn: 32701.6806492	total: 10.3s	remaining: 6m 31s
154:	learn: 32641.9524368	total: 10.4s	remaining: 6m 31s
155:	learn: 32549.3185569	total: 10.4s	remaining: 6m 31s
156:	learn: 32490.3266668	total: 10.5s	remaining: 6m 31s
157:	learn: 32423.2341727	total: 10.6s	remaining: 6m 30s
158:	learn: 32381.3498119	total: 10.6s	remaining: 6m 30s
159:	learn: 32327.9412579	total: 10.7s	remaining: 6m 30s
160:	learn: 32248.5645452	total: 10.8s	remaining: 6m 30s
161:	learn: 32197.2489163	total: 10.8s	remaining: 6m 29s
162:	learn: 32143.9614776	total: 10.9s	remaining: 6m 29s
163:	learn: 32097.7891703	total

294:	learn: 27544.4921901	total: 19.8s	remaining: 6m 23s
295:	learn: 27517.8773284	total: 19.9s	remaining: 6m 23s
296:	learn: 27490.7998288	total: 20s	remaining: 6m 23s
297:	learn: 27483.7267842	total: 20s	remaining: 6m 23s
298:	learn: 27463.3462433	total: 20.1s	remaining: 6m 23s
299:	learn: 27431.9688085	total: 20.2s	remaining: 6m 23s
300:	learn: 27423.0215332	total: 20.3s	remaining: 6m 23s
301:	learn: 27410.6316667	total: 20.3s	remaining: 6m 23s
302:	learn: 27391.7476168	total: 20.4s	remaining: 6m 23s
303:	learn: 27374.7513133	total: 20.5s	remaining: 6m 23s
304:	learn: 27356.9206009	total: 20.5s	remaining: 6m 23s
305:	learn: 27343.8381600	total: 20.6s	remaining: 6m 23s
306:	learn: 27331.5684011	total: 20.7s	remaining: 6m 23s
307:	learn: 27314.2111076	total: 20.7s	remaining: 6m 23s
308:	learn: 27288.1628702	total: 20.8s	remaining: 6m 23s
309:	learn: 27270.4888972	total: 20.9s	remaining: 6m 23s
310:	learn: 27259.3088669	total: 21s	remaining: 6m 23s
311:	learn: 27247.4616842	total: 21.1

439:	learn: 25609.0358988	total: 29.7s	remaining: 6m 15s
440:	learn: 25599.0396683	total: 29.8s	remaining: 6m 15s
441:	learn: 25586.2762453	total: 29.9s	remaining: 6m 15s
442:	learn: 25579.1523225	total: 29.9s	remaining: 6m 15s
443:	learn: 25571.7409405	total: 30s	remaining: 6m 15s
444:	learn: 25561.1848259	total: 30.1s	remaining: 6m 15s
445:	learn: 25543.9519976	total: 30.1s	remaining: 6m 15s
446:	learn: 25536.5997282	total: 30.2s	remaining: 6m 15s
447:	learn: 25529.9696810	total: 30.3s	remaining: 6m 15s
448:	learn: 25523.1970781	total: 30.3s	remaining: 6m 15s
449:	learn: 25517.8626096	total: 30.4s	remaining: 6m 15s
450:	learn: 25508.0714467	total: 30.5s	remaining: 6m 15s
451:	learn: 25501.8483109	total: 30.6s	remaining: 6m 15s
452:	learn: 25491.1872281	total: 30.6s	remaining: 6m 15s
453:	learn: 25487.5646887	total: 30.7s	remaining: 6m 14s
454:	learn: 25474.7577555	total: 30.8s	remaining: 6m 14s
455:	learn: 25460.8891818	total: 30.8s	remaining: 6m 14s
456:	learn: 25455.3951910	total: 

585:	learn: 24512.3586028	total: 41.1s	remaining: 6m 20s
586:	learn: 24507.3041828	total: 41.2s	remaining: 6m 20s
587:	learn: 24503.9141624	total: 41.3s	remaining: 6m 20s
588:	learn: 24502.0377713	total: 41.4s	remaining: 6m 20s
589:	learn: 24498.0436224	total: 41.4s	remaining: 6m 20s
590:	learn: 24489.8279403	total: 41.5s	remaining: 6m 19s
591:	learn: 24483.4833577	total: 41.6s	remaining: 6m 19s
592:	learn: 24479.5549608	total: 41.7s	remaining: 6m 19s
593:	learn: 24476.0875917	total: 41.7s	remaining: 6m 19s
594:	learn: 24474.0280242	total: 41.8s	remaining: 6m 19s
595:	learn: 24468.9659947	total: 41.9s	remaining: 6m 19s
596:	learn: 24466.5037291	total: 41.9s	remaining: 6m 19s
597:	learn: 24459.2812873	total: 42s	remaining: 6m 19s
598:	learn: 24451.7873240	total: 42.1s	remaining: 6m 19s
599:	learn: 24447.9074268	total: 42.2s	remaining: 6m 19s
600:	learn: 24444.6420447	total: 42.2s	remaining: 6m 19s
601:	learn: 24438.4703369	total: 42.3s	remaining: 6m 19s
602:	learn: 24431.6049494	total: 

...

5413:	learn: 20224.0923840	total: 7m 6s	remaining: 46.2s
5414:	learn: 20223.8973230	total: 7m 6s	remaining: 46.1s
5415:	learn: 20223.6777768	total: 7m 6s	remaining: 46s
5416:	learn: 20223.4209829	total: 7m 6s	remaining: 45.9s
5417:	learn: 20223.1901152	total: 7m 6s	remaining: 45.9s
5418:	learn: 20222.6846260	total: 7m 7s	remaining: 45.8s
5419:	learn: 20222.4309983	total: 7m 7s	remaining: 45.7s
5420:	learn: 20222.2319117	total: 7m 7s	remaining: 45.6s
5421:	learn: 20222.1977129	total: 7m 7s	remaining: 45.6s
5422:	learn: 20222.0267173	total: 7m 7s	remaining: 45.5s
5423:	learn: 20221.8192271	total: 7m 7s	remaining: 45.4s
5424:	learn: 20221.4917973	total: 7m 7s	remaining: 45.3s
5425:	learn: 20221.2065706	total: 7m 7s	remaining: 45.2s
5426:	learn: 20220.9964728	total: 7m 7s	remaining: 45.2s
5427:	learn: 20220.8001377	total: 7m 7s	remaining: 45.1s
5428:	learn: 20220.6432466	total: 7m 7s	remaining: 45s
5429:	learn: 20220.3338506	total: 7m 7s	remaining: 44.9s
5430:	learn: 20220.0633143	total: 7

5557:	learn: 20180.3254809	total: 7m 18s	remaining: 34.8s
5558:	learn: 20180.1641931	total: 7m 18s	remaining: 34.8s
5559:	learn: 20179.8590008	total: 7m 18s	remaining: 34.7s
5560:	learn: 20179.2978845	total: 7m 18s	remaining: 34.6s
5561:	learn: 20179.0104252	total: 7m 18s	remaining: 34.5s
5562:	learn: 20178.5345243	total: 7m 18s	remaining: 34.4s
5563:	learn: 20178.2737419	total: 7m 18s	remaining: 34.4s
5564:	learn: 20178.0251964	total: 7m 18s	remaining: 34.3s
5565:	learn: 20177.8486497	total: 7m 18s	remaining: 34.2s
5566:	learn: 20177.3738365	total: 7m 18s	remaining: 34.1s
5567:	learn: 20176.9145766	total: 7m 18s	remaining: 34.1s
5568:	learn: 20176.4102097	total: 7m 19s	remaining: 34s
5569:	learn: 20176.1467980	total: 7m 19s	remaining: 33.9s
5570:	learn: 20176.0379053	total: 7m 19s	remaining: 33.8s
5571:	learn: 20175.7043793	total: 7m 19s	remaining: 33.7s
5572:	learn: 20175.4679396	total: 7m 19s	remaining: 33.7s
5573:	learn: 20175.1670762	total: 7m 19s	remaining: 33.6s
5574:	learn: 201

5700:	learn: 20136.5205265	total: 7m 29s	remaining: 23.6s
5701:	learn: 20136.4005862	total: 7m 29s	remaining: 23.5s
5702:	learn: 20136.1425382	total: 7m 29s	remaining: 23.4s
5703:	learn: 20135.9106765	total: 7m 29s	remaining: 23.3s
5704:	learn: 20135.5011271	total: 7m 29s	remaining: 23.3s
5705:	learn: 20135.2082771	total: 7m 30s	remaining: 23.2s
5706:	learn: 20135.0902205	total: 7m 30s	remaining: 23.1s
5707:	learn: 20134.8189641	total: 7m 30s	remaining: 23s
5708:	learn: 20134.6402039	total: 7m 30s	remaining: 23s
5709:	learn: 20134.2998062	total: 7m 30s	remaining: 22.9s
5710:	learn: 20134.1003602	total: 7m 30s	remaining: 22.8s
5711:	learn: 20133.8838470	total: 7m 30s	remaining: 22.7s
5712:	learn: 20133.6986694	total: 7m 30s	remaining: 22.6s
5713:	learn: 20133.3810577	total: 7m 30s	remaining: 22.6s
5714:	learn: 20133.0619957	total: 7m 30s	remaining: 22.5s
5715:	learn: 20132.7648191	total: 7m 30s	remaining: 22.4s
5716:	learn: 20132.3813073	total: 7m 30s	remaining: 22.3s
5717:	learn: 20132

5843:	learn: 20095.9871959	total: 7m 41s	remaining: 12.3s
5844:	learn: 20095.7509515	total: 7m 41s	remaining: 12.2s
5845:	learn: 20095.6406865	total: 7m 41s	remaining: 12.2s
5846:	learn: 20095.2931288	total: 7m 41s	remaining: 12.1s
5847:	learn: 20095.1152347	total: 7m 41s	remaining: 12s
5848:	learn: 20094.9527250	total: 7m 41s	remaining: 11.9s
5849:	learn: 20094.9174996	total: 7m 41s	remaining: 11.8s
5850:	learn: 20094.1754583	total: 7m 41s	remaining: 11.8s
5851:	learn: 20093.9924141	total: 7m 41s	remaining: 11.7s
5852:	learn: 20093.7385868	total: 7m 41s	remaining: 11.6s
5853:	learn: 20093.4946841	total: 7m 41s	remaining: 11.5s
5854:	learn: 20093.2048445	total: 7m 41s	remaining: 11.4s
5855:	learn: 20093.0459077	total: 7m 42s	remaining: 11.4s
5856:	learn: 20092.7236196	total: 7m 42s	remaining: 11.3s
5857:	learn: 20092.5352643	total: 7m 42s	remaining: 11.2s
5858:	learn: 20092.2789428	total: 7m 42s	remaining: 11.1s
5859:	learn: 20091.8324619	total: 7m 42s	remaining: 11s
5860:	learn: 20091

5986:	learn: 20061.2250297	total: 7m 52s	remaining: 1.02s
5987:	learn: 20060.9370671	total: 7m 52s	remaining: 947ms
5988:	learn: 20060.8870386	total: 7m 52s	remaining: 868ms
5989:	learn: 20060.6355104	total: 7m 52s	remaining: 789ms
5990:	learn: 20060.5882553	total: 7m 52s	remaining: 710ms
5991:	learn: 20060.2563044	total: 7m 52s	remaining: 631ms
5992:	learn: 20059.8291470	total: 7m 52s	remaining: 552ms
5993:	learn: 20059.7094089	total: 7m 53s	remaining: 473ms
5994:	learn: 20059.4376986	total: 7m 53s	remaining: 395ms
5995:	learn: 20059.2882907	total: 7m 53s	remaining: 316ms
5996:	learn: 20059.1818710	total: 7m 53s	remaining: 237ms
5997:	learn: 20059.0384354	total: 7m 53s	remaining: 158ms
5998:	learn: 20058.9926698	total: 7m 53s	remaining: 78.9ms
5999:	learn: 20058.8600439	total: 7m 53s	remaining: 0us
--- Training took 478.67406392097473 sec ---


### Evaluation

In [8]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = mean_absolute_error(y_train, y_train_pred)
test_accuracy = mean_absolute_error(y_test, y_test_pred)

print('MAE Train: {:,}, MAE Test: {:,}'.format(
        train_accuracy, test_accuracy))

# MAE Train: 8,522.566355403891, MAE Test: 8,978.68004769769

MAE Train: 8,522.566355403891, MAE Test: 8,978.68004769769


CatBoostRegressor(iterations=6000) target encode everything
- MAE Train: 8,522.566355403891, MAE Test: 8,978.68004769769 (~8min)

## Create Submission

In [9]:
submission_data_filepath = os.path.join(DATA_DIR, 'tcd-ml-1920-group-income-test.csv')
df_submission_data = pd.read_csv(submission_data_filepath)

# Clean and encode data
df_submission_data = format_data(df_submission_data)
df_submission_data, _ = frequency_encode_data(df_submission_data, frequency_encode, frequency_encode_mappings)
df_submission_data, _ = target_encode_data(df_submission_data, target_encode, target_encode_mappings)

# TODO make func
# Standardize data
df_submission_data[scale_columns] = scaler.fit_transform(df_submission_data[scale_columns])

# Make predictions
X = df_submission_data.drop(columns=TARGET_VARIABLE).values
y_pred = model.predict(X)

# Create submission
df_submission_filepath = os.path.join(DATA_DIR, 'tcd-ml-1920-group-income-submission.csv')
df_submission = pd.read_csv(df_submission_filepath)
df_submission['Total Yearly Income [EUR]'] = pd.Series(y_pred)
submission_filepath = os.path.join(DATA_DIR, 'submission', 'submission.csv')
df_submission.to_csv(submission_filepath, index=False)



c:\users\fitze\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
